In [15]:
start_date = '2016-01-01'
end_date = '2021-06-01'
target = "10d_ret_standscalar"
alpha_name = "Original_Input_Ts_Stand_Residual_Concated"
shift = 1
sequence = 10
LR = [0.001,0.0001,0.00001]
epoch_num = [30,30,30]

In [12]:
feat_list = alpha_list.copy()
# method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]
# from AlphaNet.Data import generate_alpha_list
# alpha_list = generate_alpha_list(feat_list,method,day=10)
alpha_list = [i +"_residual" for i in feat_list]

In [14]:
alpha_list

['open_residual',
 'high_residual',
 'low_residual',
 'close_residual',
 'vwap_residual',
 'volume_residual',
 'pct_chg_residual',
 'turnover_residual',
 'free_turnover_residual',
 'open_ts_norm_30d',
 'high_ts_norm_30d',
 'low_ts_norm_30d',
 'close_ts_norm_30d',
 'vwap_ts_norm_30d',
 'volume_ts_norm_30d',
 'pct_chg_ts_norm_30d',
 'turnover_ts_norm_30d',
 'free_turnover_ts_norm_30d']

In [16]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import concat_original_data , generate_shift_data

In [17]:
concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
['open_residual', 'high_residual', 'low_residual', 'close_residual', 'vwap_residual', 'volume_residual', 'pct_chg_residual', 'turnover_residual', 'free_turnover_residual', 'open_ts_norm_30d', 'high_ts_norm_30d', 'low_ts_norm_30d', 'close_ts_norm_30d', 'vwap_ts_norm_30d', 'volume_ts_norm_30d', 'pct_chg_ts_norm_30d', 'turnover_ts_norm_30d', 'free_turnover_ts_norm_30d']


100%|██████████| 1315/1315 [00:28<00:00, 45.48it/s]


open_residual  high_residual  low_residual  \
timestamp           ticker                                               
2016-01-04 09:30:00 600652      21.719064      21.725644     21.677728   
                    600651      21.346758      21.370448     21.330183   
                    600653      20.927884      20.929687     20.898577   
                    600601      20.509053      20.510626     20.478675   
                    600602      18.780087      18.836678     18.775355   
...                                   ...            ...           ...   
2021-05-28 09:30:00 688198     -15.221922     -15.238036    -15.137327   
                    300346     -15.472607     -15.480389    -15.464044   
                    688029     -15.577614     -15.667533    -15.577614   
                    300841     -16.171659     -16.175826    -16.144186   
                    300059     -17.484425     -17.490150    -17.459840   

                            close_residual  vwap_residual  volume_residual  \
timestamp           ticker                                                   
2016-01-04 09:30:00 600652       21.678645      21.695113        22.185825   
                    600651       21.356980      21.352795        22.267718   
                    600653       20.907828      20.917122        22.067467   
                    600601       20.481916      20.495353        22.834911   
                    600602       18.786362      18.806218        22.653143   
...                                    ...            ...              ...   
2021-05-28 09:30:00 688198      -15.160606     -15.187998       -18.560362   
                    300346      -15.464425     -15.471517       -20.996563   
                    688029      -15.648587     -15.638958       -19.465924   
                    300841      -16.165418     -16.161042       -18.341441   
                    300059      -17.466508     -17.471152       -24.700989   

                            pct_chg_residual  turnover_residual  \
timestamp           ticker                                        
2016-01-04 09:30:00 600652          0.000000          11.156315   
                    600651         10.211395          11.017944   
                    600653          0.000000           9.996995   
                    600601          0.000000          10.535850   
                    600602         10.236494          11.268524   
...                                      ...                ...   
2021-05-28 09:30:00 688198          0.000000         -10.828421   
                    300346          0.000000         -10.458437   
                    688029        -11.876606         -11.103069   
                    300841          0.000000         -11.028226   
                    300059          0.000000         -11.223452   

                            free_turnover_residual  open_ts_norm_30d  \
timestamp           ticker                                             
2016-01-04 09:30:00 600652               11.258092               NaN   
                    600651               11.265284               NaN   
                    600653               10.149023               NaN   
                    600601               10.659694               NaN   
                    600602               11.781086               NaN   
...                                            ...               ...   
2021-05-28 09:30:00 688198              -10.828421               NaN   
                    300346              -10.723628               NaN   
                    688029              -11.191070               NaN   
                    300841              -11.028226               NaN   
                    300059              -11.336920               NaN   

                            high_ts_norm_30d  low_ts_norm_30d  \
timestamp           ticker                                      
2016-01-04 09:30:00 600652               NaN              NaN   
                    600651               NaN  

In [18]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target)

100%|██████████| 1705/1705 [00:16<00:00, 101.71it/s]


[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


100%|██████████| 1277/1277 [02:14<00:00,  9.51it/s]


open_residual_Shift_9  high_residual_Shift_9  \
timestamp           ticker                                                 
2016-03-03 09:30:00 000001               0.014845               0.000000   
                    000004               0.007484               0.003363   
                    000005               0.011579               0.021006   
                    000006               0.036322               0.033223   
                    000008               0.022732              -0.005572   
...                                           ...                    ...   
2021-05-31 09:30:00 688689               0.038435               0.037507   
                    688699              14.930076              14.992200   
                    688777               0.012987               0.021771   
                    688788               0.023793               0.059120   
                    688981              -0.019231              -0.010231   

                            low_residual_Shift_9  close_residual_Shift_9  \
timestamp           ticker                                                 
2016-03-03 09:30:00 000001              0.009961               -0.005929   
                    000004              0.009455               -0.011798   
                    000005              0.014491               -0.002500   
                    000006              0.035504                0.004561   
                    000008              0.004631               -0.012553   
...                                          ...                     ...   
2021-05-31 09:30:00 688689              0.024195                0.026949   
                    688699             14.914271               14.918301   
                    688777              0.029063                0.010064   
                    688788              0.029756                0.032484   
                    688981             -0.009941               -0.014062   

                            vwap_residual_Shift_9  volume_residual_Shift_9  \
timestamp           ticker                                                   
2016-03-03 09:30:00 000001               0.005908                -0.365105   
                    000004               0.004482                -0.007508   
                    000005               0.014790                 0.007685   
                    000006               0.036116                 0.089657   
                    000008              -0.005474                -0.042390   
...                                           ...                      ...   
2021-05-31 09:30:00 688689               0.032691                 0.424874   
                    688699              14.954830                19.972331   
                    688777               0.020765                -0.236341   
                    688788               0.040894                 0.735762   
                    688981              -0.007632                -0.245412   

                            pct_chg_residual_Shift_9  \
timestamp           ticker                             
2016-03-03 09:30:00 000001                -10.335472   
                    000004                 -9.575658   
                    000005                -10.038259   
                    000006                 -2.116801   
                    000008                -10.479211   
...                                              ...   
2021-05-31 09:30:00 688689                  0.890756   
                    688699                  9.848363   
                    688777                 -0.545721   
                    688788                  0.144210   
                    688981                 -7.364735   

                            turnover_residual_Shift_9  \
timestamp           ticker                              
2016-03-03 09:30:00 000001                  -0.365039   
                    000004                  -0.007503   
                    000005                   0.007678   
                    0

In [19]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
display(original)
# Timelist
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])
display(time_list2)

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
...,...,...,...,...,...,...,...,...,...
70,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
71,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
72,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


,start_date,end_date
0,2019-01-01,2019-06-01
1,2019-06-01,2020-01-01
2,2020-01-01,2020-06-01
3,2020-06-01,2021-01-01
4,2021-01-01,2021-06-01


In [20]:
task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i" % (shift,sequence),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
0,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
1,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
2,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
3,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
4,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [21]:
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
task[-20:]

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
60,AlphaNetV1_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
61,AlphaNetV1_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
62,AlphaNetV1_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
63,AlphaNetV1_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
64,AlphaNetV1_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
65,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
66,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
67,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
68,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [22]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)